Copyright 2020 Google LLC.

Licensed aunder the Apache License, Version 2.0 (the "License");

In [ ]:
#@title License
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RepNet

This colab contains a pre-trained [RepNet](https://sites.google.com/view/repnet) model. It can be used to predict the rate at which repetitions are happening in a video in a class-agnostic manner. These estimates can be used to count the number of repetitions in videos.

This model is able to count repetitions in many domains: counting the number of reps while exercising, measuring the rate of biological events like heartrates etc. 

Ensure you are running the Colab with a GPU runtime.


# Setup



In [1]:
#!pip3 install opencv-python

In [1]:
print("hello")

hello


In [9]:
# !pip3 install pillow

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [16]:
# !pip3 uninstall tensorflow==1.13.1

In [1]:
# !pip3 install -U "tensorflow==1.15" 

  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7540 sha256=9b19627d1d39365d52e52016e9b6a46ca724538e7eccdc4445d2f65ea5c5c775
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\c4\b2\f9\b3052fd0a0c1e61f4eb5b879161a8b6670fb1c26951a5ad5d6
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.5
    Uninstalling numpy-1.14.5:
      Successfully uninstalled numpy-1.14.5
  Attempting uninstall: setuptools
    Found existing installation: setuptools 39.1.0
    Uninstalling setuptools-39.1.0:
      Successfully uninstalled setuptools-39.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [11]:
# !pip3 install scipy

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [6]:
# !pip3 install  matplotlib

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [2]:
import matplotlib.animation as animation
from IPython.display import HTML

In [3]:
#@title
# import matplotlib.animation as animation
# from IPython.display import HTML
import base64
import io
import os
# import time

import cv2

# from IPython.display import display
# from IPython.display import HTML
# from IPython.display import Javascript

# import matplotlib
# from matplotlib.animation import FuncAnimation
# import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
from scipy.signal import medfilt
import tensorflow as tf
# import tensorflow.compat.v2 as tf
# tf.compat.v1.enable_v2_behavior()
# ! pip install youtube_dl
# import youtube_dl

# from google.colab import drive
# from google.colab import output
# from google.colab.output import eval_js

# Model definition
layers = tf.keras.layers
regularizers = tf.keras.regularizers


class ResnetPeriodEstimator(tf.keras.models.Model):
  """RepNet model."""

  def __init__(
      self,
      num_frames=64,
      image_size=112,
      base_model_layer_name='conv4_block3_out',
      temperature=13.544,
      dropout_rate=0.25,
      l2_reg_weight=1e-6,
      temporal_conv_channels=512,
      temporal_conv_kernel_size=3,
      temporal_conv_dilation_rate=3,
      conv_channels=32,
      conv_kernel_size=3,
      transformer_layers_config=((512, 4, 512),),
      transformer_dropout_rate=0.0,
      transformer_reorder_ln=True,
      period_fc_channels=(512, 512),
      within_period_fc_channels=(512, 512)):
    super(ResnetPeriodEstimator, self).__init__()

    # Model params.
    self.num_frames = num_frames
    self.image_size = image_size

    self.base_model_layer_name = base_model_layer_name

    self.temperature = temperature

    self.dropout_rate = dropout_rate
    self.l2_reg_weight = l2_reg_weight

    self.temporal_conv_channels = temporal_conv_channels
    self.temporal_conv_kernel_size = temporal_conv_kernel_size
    self.temporal_conv_dilation_rate = temporal_conv_dilation_rate

    self.conv_channels = conv_channels
    self.conv_kernel_size = conv_kernel_size
    # Transformer config in form of (channels, heads, bottleneck channels).
    self.transformer_layers_config = transformer_layers_config
    self.transformer_dropout_rate = transformer_dropout_rate
    self.transformer_reorder_ln = transformer_reorder_ln

    self.period_fc_channels = period_fc_channels
    self.within_period_fc_channels = within_period_fc_channels

    # Base ResNet50 Model.
    base_model = tf.keras.applications.ResNet50V2(
        include_top=False, weights=None, pooling='max')
    self.base_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=base_model.get_layer(self.base_model_layer_name).output)

    # 3D Conv on k Frames
    self.temporal_conv_layers = [
        layers.Conv3D(self.temporal_conv_channels,
                      self.temporal_conv_kernel_size,
                      padding='same',
                      dilation_rate=(self.temporal_conv_dilation_rate, 1, 1),
                      kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                      kernel_initializer='he_normal')]
    self.temporal_bn_layers = [layers.BatchNormalization()
                               for _ in self.temporal_conv_layers]

    # Counting Module (Self-sim > Conv > Transformer > Classifier)
    self.conv_3x3_layer = layers.Conv2D(self.conv_channels,
                                        self.conv_kernel_size,
                                        padding='same',
                                        activation=tf.nn.relu)

    channels = self.transformer_layers_config[0][0]
    self.input_projection = layers.Dense(
        channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
        activation=None)
    self.input_projection2 = layers.Dense(
        channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
        activation=None)

    length = self.num_frames
    self.pos_encoding = tf.compat.v1.get_variable(
        name='resnet_period_estimator/pos_encoding',
        shape=[1, length, 1],
        initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))
    self.pos_encoding2 = tf.compat.v1.get_variable(
        name='resnet_period_estimator/pos_encoding2',
        shape=[1, length, 1],
        initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))

    self.transformer_layers = []
    for d_model, num_heads, dff in self.transformer_layers_config:
      self.transformer_layers.append(
          TransformerLayer(d_model, num_heads, dff,
                           self.transformer_dropout_rate,
                           self.transformer_reorder_ln))

    self.transformer_layers2 = []
    for d_model, num_heads, dff in self.transformer_layers_config:
      self.transformer_layers2.append(
          TransformerLayer(d_model, num_heads, dff,
                           self.transformer_dropout_rate,
                           self.transformer_reorder_ln))

    # Period Prediction Module.
    self.dropout_layer = layers.Dropout(self.dropout_rate)
    num_preds = self.num_frames//2
    self.fc_layers = []
    for channels in self.period_fc_channels:
      self.fc_layers.append(layers.Dense(
          channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
          activation=tf.nn.relu))
    self.fc_layers.append(layers.Dense(
        num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight)))

    # Within Period Module
    num_preds = 1
    self.within_period_fc_layers = []
    for channels in self.within_period_fc_channels:
      self.within_period_fc_layers.append(layers.Dense(
          channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
          activation=tf.nn.relu))
    self.within_period_fc_layers.append(layers.Dense(
        num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight)))

  def call(self, x):
    # Ensures we are always using the right batch_size during train/eval.
    batch_size = tf.shape(x)[0]
    # Conv Feature Extractor.
    x = tf.reshape(x, [-1, self.image_size, self.image_size, 3])
    x = self.base_model(x)
    h = tf.shape(x)[1]
    w = tf.shape(x)[2]
    c = tf.shape(x)[3]
    x = tf.reshape(x, [batch_size, -1, h, w, c])

    # 3D Conv to give temporal context to per-frame embeddings. 
    for bn_layer, conv_layer in zip(self.temporal_bn_layers,
                                    self.temporal_conv_layers):
      x = conv_layer(x)
      x = bn_layer(x)
      x = tf.nn.relu(x)

    x = tf.reduce_max(x, [2, 3])

    # Reshape and prepare embs for output.
    final_embs = x

    # Get self-similarity matrix.
    x = get_sims(x, self.temperature)

    # 3x3 conv layer on self-similarity matrix.
    x = self.conv_3x3_layer(x)
    x = tf.reshape(x, [batch_size, self.num_frames, -1])
    within_period_x = x

    # Period prediction.
    x = self.input_projection(x)
    x += self.pos_encoding
    for transformer_layer in self.transformer_layers:
      x = transformer_layer(x)
    x = flatten_sequential_feats(x, batch_size, self.num_frames)
    for fc_layer in self.fc_layers:
      x = self.dropout_layer(x)
      x = fc_layer(x)

    # Within period prediction.
    within_period_x = self.input_projection2(within_period_x)
    within_period_x += self.pos_encoding2
    for transformer_layer in self.transformer_layers2:
      within_period_x = transformer_layer(within_period_x)
    within_period_x = flatten_sequential_feats(within_period_x,
                                               batch_size,
                                               self.num_frames)
    for fc_layer in self.within_period_fc_layers:
      within_period_x = self.dropout_layer(within_period_x)
      within_period_x = fc_layer(within_period_x)

    return x, within_period_x, final_embs

  @tf.function
  def preprocess(self, imgs):
    imgs = tf.cast(imgs, tf.float32)
    imgs -= 127.5
    imgs /= 127.5
    imgs = tf.image.resize(imgs, (self.image_size, self.image_size))
    return imgs


def get_sims(embs, temperature):
  """Calculates self-similarity between batch of sequence of embeddings."""
  batch_size = tf.shape(embs)[0]
  seq_len = tf.shape(embs)[1]
  embs = tf.reshape(embs, [batch_size, seq_len, -1])

  def _get_sims(embs):
    """Calculates self-similarity between sequence of embeddings."""
    dist = pairwise_l2_distance(embs, embs)
    sims = -1.0 * dist
    return sims

  sims = tf.map_fn(_get_sims, embs)
  sims /= temperature
  sims = tf.nn.softmax(sims, axis=-1)
  sims = tf.expand_dims(sims, -1)
  return sims


def flatten_sequential_feats(x, batch_size, seq_len):
  """Flattens sequential features with known batch size and seq_len."""
  x = tf.reshape(x, [batch_size, seq_len, -1])
  return x


# Transformer from https://www.tensorflow.org/tutorials/text/transformer .
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.

  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    outputs: shape == (..., seq_len_q, depth_v)
    attention_weights: shape == (..., seq_len_q, seq_len_k)
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk.
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  # (..., seq_len_q, seq_len_k)
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

  outputs = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return outputs, attention_weights


def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])


class MultiHeadAttention(tf.keras.layers.Layer):
  """Multi-headed attention layer."""

  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth)."""
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(
        q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(
        k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(
        v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(
        scaled_attention,
        perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(
        scaled_attention,
        (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    outputs = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return outputs, attention_weights


class TransformerLayer(tf.keras.layers.Layer):
  """Implements a single transformer layer (https://arxiv.org/abs/1706.03762).
  """

  def __init__(self, d_model, num_heads, dff,
               dropout_rate=0.1,
               reorder_ln=False):
    super(TransformerLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.reorder_ln = reorder_ln

  def call(self, x):
    inp_x = x

    if self.reorder_ln:
      x = self.layernorm1(x)

    # (batch_size, input_seq_len, d_model)
    attn_output, _ = self.mha(x, x, x, mask=None)
    attn_output = self.dropout1(attn_output)

    if self.reorder_ln:
      out1 = inp_x + attn_output
      x = out1
    else:
      # (batch_size, input_seq_len, d_model)
      out1 = self.layernorm1(x + attn_output)
      x = out1

    if self.reorder_ln:
      x = self.layernorm2(x)

    # (batch_size, input_seq_len, d_model)
    ffn_output = self.ffn(x)
    ffn_output = self.dropout2(ffn_output)

    if self.reorder_ln:
      out2 = out1 + ffn_output
    else:
      # (batch_size, input_seq_len, d_model)
      out2 = self.layernorm2(out1 + ffn_output)

    return out2


def pairwise_l2_distance(a, b):
  """Computes pairwise distances between all rows of a and all rows of b."""
  norm_a = tf.reduce_sum(tf.square(a), 1)
  norm_a = tf.reshape(norm_a, [-1, 1])
  norm_b = tf.reduce_sum(tf.square(b), 1)
  norm_b = tf.reshape(norm_b, [1, -1])
  dist = tf.maximum(norm_a - 2.0 * tf.matmul(a, b, False, True) + norm_b, 0.0)
  return dist


def get_repnet_model(logdir):
  """Returns a trained RepNet model.

  Args:
    logdir (string): Path to directory where checkpoint will be downloaded.

  Returns:
    model (Keras model): Trained RepNet model.
  """
  # Check if we are in eager mode.
  
  assert tf.executing_eagerly()
  
  
  # Models will be called in eval mode.
  tf.keras.backend.set_learning_phase(0)

  # Define RepNet model.
  model = ResnetPeriodEstimator()
  # tf.function for speed.
  model.call = tf.function(model.call)

  # Define checkpoint and checkpoint manager.
  ckpt = tf.train.Checkpoint(model=model)
  ckpt_manager = tf.train.CheckpointManager(
      ckpt, directory=logdir, max_to_keep=10)
  latest_ckpt = ckpt_manager.latest_checkpoint
  print('Loading from: ', latest_ckpt)
  if not latest_ckpt:
    raise ValueError('Path does not have a checkpoint to load.')
  # Restore weights.
  ckpt.restore(latest_ckpt).expect_partial()

  # Pass dummy frames to build graph.
  model(tf.random.uniform((1, 64, 112, 112, 3)))
  return model


def unnorm(query_frame):
  min_v = query_frame.min()
  max_v = query_frame.max()
  query_frame = (query_frame - min_v) / max(1e-7, (max_v - min_v))
  return query_frame


# def create_count_video(frames,
#                        per_frame_counts,
#                        within_period,
#                        score,
#                        fps,
#                        output_file,
#                        delay,
#                        plot_count=True,
#                        plot_within_period=False,
#                        plot_score=False):
#   """Creates video with running count and within period predictions.

#   Args:
#     frames (List): List of images in form of NumPy arrays.
#     per_frame_counts (List): List of floats indicating repetition count for
#       each frame. This is the rate of repetition for that particular frame.
#       Summing this list up gives count over entire video.
#     within_period (List): List of floats indicating score between 0 and 1 if the
#       frame is inside the periodic/repeating portion of a video or not.
#     score (float): Score between 0 and 1 indicating the confidence of the
#       RepNet model's count predictions.
#     fps (int): Frames per second of the input video. Used to scale the
#       repetition rate predictions to Hz.
#     output_file (string): Path of the output video.
#     delay (integer): Delay between each frame in the output video.
#     plot_count (boolean): if True plots the count in the output video.
#     plot_within_period (boolean): if True plots the per-frame within period
#       scores.
#     plot_score (boolean): if True plots the confidence of the model along with
#       count ot within_period scores.
#   """
#   if output_file[-4:] not in ['.mp4', '.gif']:
#     raise ValueError('Output format can only be mp4 or gif')
#   num_frames = len(frames)

#   running_counts = np.cumsum(per_frame_counts)
#   final_count = running_counts[-1]
#   print(final_count)
#   def count(idx):
#     return int(np.round(running_counts[idx]))

#   def rate(idx):
#     return per_frame_counts[idx] * fps

#   if plot_count and not plot_within_period:
#     fig = plt.figure(figsize=(10, 12), tight_layout=True)
#     im = plt.imshow(unnorm(frames[0]))
#     if plot_score:
#       plt.suptitle('Pred Count: %d, '
#                    'Prob: %0.1f' % (int(np.around(final_count)), score),
#                    fontsize=24)

#     plt.title('Count 0, Rate: 0', fontsize=24)
#     plt.axis('off')
#     plt.grid(b=None)
#     def update_count_plot(i):
#       """Updates the count plot."""
#       im.set_data(unnorm(frames[i]))
#       plt.title('Count %d, Rate: %0.4f Hz' % (count(i), rate(i)), fontsize=24)

#     anim = FuncAnimation(
#         fig,
#         update_count_plot,
#         frames=np.arange(1, num_frames),
#         interval=delay,
#         blit=False)
#     if output_file[-3:] == 'mp4':
#       anim.save(output_file, dpi=100, fps=24)
#     elif output_file[-3:] == 'gif':
#       anim.save(output_file, writer='imagemagick', fps=24, dpi=100)

#   elif plot_within_period:
#     fig, axs = plt.subplots(1, 2, figsize=(12, 6))
#     im = axs[0].imshow(unnorm(frames[0]))
#     axs[1].plot(0, within_period[0])
#     axs[1].set_xlim((0, len(frames)))
#     axs[1].set_ylim((0, 1))

#     if plot_score:
#       plt.suptitle('Pred Count: %d, '
#                    'Prob: %0.1f' % (int(np.around(final_count)), score),
#                    fontsize=24)

#     if plot_count:
#       axs[0].set_title('Count 0, Rate: 0', fontsize=20)

#     plt.axis('off')
#     plt.grid(b=None)

#     def update_within_period_plot(i):
#       """Updates the within period plot along with count."""
#       im.set_data(unnorm(frames[i]))
#       axs[0].set_xticks([])
#       axs[0].set_yticks([])
#       xs = []
#       ys = []
#       if plot_count:
#         axs[0].set_title('Count %d, Rate: %0.4f Hz' % (count(i), rate(i)),
#                          fontsize=20)
#       for idx in range(i):
#         xs.append(idx)
#         ys.append(within_period[int(idx * len(within_period) / num_frames)])
#       axs[1].clear()
#       axs[1].set_title('Within Period or Not', fontsize=20)
#       axs[1].set_xlim((0, num_frames))
#       axs[1].set_ylim((-0.05, 1.05))
#       axs[1].plot(xs, ys)

#     anim = FuncAnimation(
#         fig,
#         update_within_period_plot,
#         frames=np.arange(1, num_frames),
#         interval=delay,
#         blit=False,
#     )
#     if output_file[-3:] == 'mp4':
#       anim.save(output_file, dpi=100, fps=24)
#     elif output_file[-3:] == 'gif':
#       anim.save(output_file, writer='imagemagick', fps=24, dpi=100)

#   plt.close()


# def show_video(video_path):
#   mp4 = open(video_path, 'rb').read()
#   data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
#   return HTML("""<video width=600 controls>
#       <source src="%s" type="video/mp4"></video>
#   """ % data_url)




# def viz_reps(frames,
#              count,
#              score,
#              name,
#              alpha=1.0,
#              pichart=True,
#              colormap=plt.cm.PuBu,
#              num_frames=None,
#              interval=30,
#              plot_score=True):
#   """Visualize repetitions."""
#   if isinstance(count, list):
#     counts = len(frames) * [count/len(frames)]
#   else:
#     counts = count
#   sum_counts = np.cumsum(counts)
#   # print(sum_counts[-1])
#   tmp_path = r"C:\Users\lenovo\Desktop\RepNet\tmp\output_%s.mp4"%name
#   fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(5, 5),
#                          tight_layout=True,)
#   fig.set_tight_layout(False)
#   h, w, _ = np.shape(frames[0])
#   wedge_x = 95 / 112 * w
#   wedge_y = 17 / 112 * h
#   wedge_r = 15 / 112 * h
#   txt_x = 95 / 112 * w
#   txt_y = 19 / 112 * h
#   otxt_size = 62 / 112 * h

#   if plot_score:
#     plt.title('Score:%.2f' % score, fontsize=20)
#   im0 = ax.imshow(unnorm(frames[0]))

#   if not num_frames:
#     num_frames = len(frames)

#   if pichart:
#     wedge1 = matplotlib.patches.Wedge(
#         center=(wedge_x, wedge_y),
#         r=wedge_r,
#         theta1=0,
#         theta2=0,
#         color=colormap(1.),
#         alpha=alpha)
#     wedge2 = matplotlib.patches.Wedge(
#         center=(wedge_x, wedge_y),
#         r=wedge_r,
#         theta1=0,
#         theta2=0,
#         color=colormap(0.5),
#         alpha=alpha)

#     ax.add_patch(wedge1)
#     ax.add_patch(wedge2)
#     txt = ax.text(
#         txt_x,
#         txt_y,
#         '0',
#         size=35,
#         ha='center',
#         va='center',
#         alpha=0.9,
#         color='white',
#     )

#   else:
#     txt = ax.text(
#         txt_x,
#         txt_y,
#         '0',
#         size=otxt_size,
#         ha='center',
#         va='center',
#         alpha=0.8,
#         color=colormap(0.4),
#     )

#   def update(i):
#     """Update plot with next frame."""
#     im0.set_data(unnorm(frames[i]))
#     ctr = int(sum_counts[i])
#     if pichart:
#       if ctr%2 == 0:
#         wedge1.set_color(colormap(1.0))
#         wedge2.set_color(colormap(0.5))
#       else:
#         wedge1.set_color(colormap(0.5))
#         wedge2.set_color(colormap(1.0))

#       wedge1.set_theta1(-90)
#       wedge1.set_theta2(-90 - 360 * (1 - sum_counts[i] % 1.0))
#       wedge2.set_theta1(-90 - 360 * (1 - sum_counts[i] % 1.0))
#       wedge2.set_theta2(-90)

#     txt.set_text(int(sum_counts[i]))
#     ax.grid(False)
#     ax.set_xticks([])
#     ax.set_yticks([])
#     plt.tight_layout()
    

#   anim = FuncAnimation(
#       fig,
#       update,
#       frames=num_frames,
#       interval=interval,
#       blit=False)
#   anim.save(tmp_path, dpi=80)
#   plt.close()
#   return show_video(tmp_path)


# def record_video(interval_in_ms, num_frames, quality=0.8):
#   """Capture video from webcam."""
#   # https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb.

#   # Give warning before recording.
#   for i in range(0, 3):
#     print('Opening webcam in %d seconds'%(3-i))
#     time.sleep(1)
#     output.clear('status_text')

#   js = Javascript('''
#     async function recordVideo(interval_in_ms, num_frames, quality) {
#       const div = document.createElement('div');
#       const video = document.createElement('video');
#       video.style.display = 'block';
#       const stream = await navigator.mediaDevices.getUserMedia({video: true});

#       // show the video in the HTML element
#       document.body.appendChild(div);
#       div.appendChild(video);

#       video.srcObject = stream;
#       await video.play();

#       google.colab.output.setIframeHeight(document.documentElement.scrollHeight,
#         true);

#       for (let i = 0; i < num_frames; i++) {
#         const canvas = document.createElement('canvas');
#         canvas.width = video.videoWidth;
#         canvas.height = video.videoHeight;
#         canvas.getContext('2d').drawImage(video, 0, 0);
#         img = canvas.toDataURL('image/jpeg', quality);
#         google.colab.kernel.invokeFunction(
#         'notebook.get_webcam_video', [img], {});
#         await new Promise(resolve => setTimeout(resolve, interval_in_ms));
#       }
#       stream.getVideoTracks()[0].stop();
#       div.remove();
#     }
#     ''')
#   display(js)
#   eval_js('recordVideo({},{},{})'.format(interval_in_ms, num_frames, quality))


def data_uri_to_img(uri):
  """Convert base64image to Numpy array."""
  image = base64.b64decode(uri.split(',')[1], validate=True)
  # Binary string to PIL image.
  image = Image.open(io.BytesIO(image))
  image = image.resize((224, 224))
  # PIL to Numpy array.
  image = np.array(np.array(image, dtype=np.uint8), np.float32)
  return image


def read_video(video_filename, width=224, height=224):
  """Read video from file."""
  cap = cv2.VideoCapture(video_filename)
  fps = cap.get(cv2.CAP_PROP_FPS)
  frames = []
  if cap.isOpened():
    while True:
      success, frame_bgr = cap.read()
      if not success:
        break
      frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
      frame_rgb = cv2.resize(frame_rgb, (width, height))
      frames.append(frame_rgb)
  frames = np.asarray(frames)
  return frames, fps


# def get_webcam_video(img_b64):
#   """Populates global variable imgs by converting image URI to Numpy array."""
#   image = data_uri_to_img(img_b64)
#   imgs.append(image)


# def download_video_from_url(url_to_video,
#                             path_to_video='/tmp/video.mp4'):
#   if os.path.exists(path_to_video):
#     os.remove(path_to_video)
#   ydl_opts = {
#       'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
#       'outtmpl': str(path_to_video),
#   }
#   with youtube_dl.YoutubeDL(ydl_opts) as ydl:
#     ydl.download([url_to_video])


def get_score(period_score, within_period_score):
  """Combine the period and periodicity scores."""
  within_period_score = tf.nn.sigmoid(within_period_score)[:, 0]
  per_frame_periods = tf.argmax(period_score, axis=-1) + 1
  pred_period_conf = tf.reduce_max(
      tf.nn.softmax(period_score, axis=-1), axis=-1)
  pred_period_conf = np.where(
      tf.math.less(per_frame_periods, 3), 0.0, pred_period_conf)#original is tf
  within_period_score *= pred_period_conf
  within_period_score = np.sqrt(within_period_score)
  pred_score = tf.reduce_mean(within_period_score)
  return pred_score, within_period_score


def get_counts(model, frames, strides, batch_size,
               threshold,
               within_period_threshold,
               constant_speed=False,
               median_filter=False,
               fully_periodic=False):
  """Pass frames through model and conver period predictions to count."""
  seq_len = len(frames)
  raw_scores_list = []
  scores = []
  within_period_scores_list = []

  if fully_periodic:
    within_period_threshold = 0.0

  frames = model.preprocess(frames)

  for stride in strides:
    num_batches = int(np.ceil(seq_len/model.num_frames/stride/batch_size))
    raw_scores_per_stride = []
    within_period_score_stride = []
    for batch_idx in range(num_batches):
      idxes = tf.range(batch_idx*batch_size*model.num_frames*stride,
                       (batch_idx+1)*batch_size*model.num_frames*stride,
                       stride)
      idxes = tf.clip_by_value(idxes, 0, seq_len-1)
      curr_frames = tf.gather(frames, idxes)
      curr_frames = tf.reshape(
          curr_frames,
          [batch_size, model.num_frames, model.image_size, model.image_size, 3])

      raw_scores, within_period_scores, _ = model(curr_frames)
      raw_scores_per_stride.append(np.reshape(raw_scores.numpy(),
                                              [-1, model.num_frames//2]))
      within_period_score_stride.append(np.reshape(within_period_scores.numpy(),
                                                   [-1, 1]))
    raw_scores_per_stride = np.concatenate(raw_scores_per_stride, axis=0)
    raw_scores_list.append(raw_scores_per_stride)
    within_period_score_stride = np.concatenate(
        within_period_score_stride, axis=0)
    pred_score, within_period_score_stride = get_score(
        raw_scores_per_stride, within_period_score_stride)
    scores.append(pred_score)
    within_period_scores_list.append(within_period_score_stride)

  # Stride chooser
  argmax_strides = np.argmax(scores)
  chosen_stride = strides[argmax_strides]
  raw_scores = np.repeat(
      raw_scores_list[argmax_strides], chosen_stride, axis=0)[:seq_len]
  within_period = np.repeat(
      within_period_scores_list[argmax_strides], chosen_stride,
      axis=0)[:seq_len]
  within_period_binary = np.asarray(within_period > within_period_threshold)
  if median_filter:
    within_period_binary = medfilt(within_period_binary, 5)

  # Select Periodic frames
  periodic_idxes = np.where(within_period_binary)[0]

  if constant_speed:
    # Count by averaging predictions. Smoother but
    # assumes constant speed.
    scores = tf.reduce_mean(
        tf.nn.softmax(raw_scores[periodic_idxes], axis=-1), axis=0)
    max_period = np.argmax(scores)
    pred_score = scores[max_period]
    pred_period = chosen_stride * (max_period + 1)
    per_frame_counts = (
        np.asarray(seq_len * [1. / pred_period]) *
        np.asarray(within_period_binary))
  else:
    # Count each frame. More noisy but adapts to changes in speed.
    pred_score = tf.reduce_mean(within_period)
    per_frame_periods = tf.argmax(raw_scores, axis=-1) + 1
    per_frame_counts = np.where(
        tf.math.less(per_frame_periods, 3),
        0.0,
        tf.math.divide(1.0,
                       tf.cast(chosen_stride * per_frame_periods, tf.float32)),
    )
    if median_filter:
      per_frame_counts = medfilt(per_frame_counts, 5)

    per_frame_counts *= np.asarray(within_period_binary)

    pred_period = seq_len/np.sum(per_frame_counts)

  if pred_score < threshold:
    print('No repetitions detected in video as score '
          '%0.2f is less than threshold %0.2f.'%(pred_score, threshold))
    per_frame_counts = np.asarray(len(per_frame_counts) * [0.])

  return (pred_period, pred_score, within_period,
          per_frame_counts, chosen_stride)

## Load trained RepNet

In [4]:
PATH_TO_CKPT = r'E:\itsspm\itsspm\modelcheckpoints\repnet_ckpt'
tf.compat.v1.enable_v2_behavior()

# !mkdir $PATH_TO_CKPT
# !wget -nc -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/checkpoint
# !wget -nc -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00000-of-00002
# !wget -nc -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00001-of-00002
# !wget -nc -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.index
# !wget -nc -P $'C:\Users\lenovo\Desktop\RepNet\tmp\repnet_ckpt' https://storage.googleapis.com/repnet_ckpt/checkpoint
model = get_repnet_model(PATH_TO_CKPT)

Loading from:  E:\itsspm\itsspm\modelcheckpoints\repnet_ckpt\ckpt-88


## Set Params



In [5]:
##@title 

# FPS while recording video from webcam.
WEBCAM_FPS = 16#@param {type:"integer"}

# Time in seconds to record video on webcam. 
RECORDING_TIME_IN_SECONDS = 8. #@param {type:"number"}

# Threshold to consider periodicity in entire video.
THRESHOLD = 0.001#@param {type:"number"}

# Threshold to consider periodicity for individual frames in video.
WITHIN_PERIOD_THRESHOLD = 0.005#@param {type:"number"}

# Use this setting for better results when it is 
# known action is repeating at constant speed.
CONSTANT_SPEED = False#@param {type:"boolean"}

# Use median filtering in time to ignore noisy frames.
MEDIAN_FILTER = True#@param {type:"boolean"}

# Use this setting for better results when it is 
# known the entire video is periodic/reapeating and
# has no aperiodic frames.
FULLY_PERIODIC = True#@param {type:"boolean"}

# Plot score in visualization video.
PLOT_SCORE = False#@param {type:"boolean"}

# Visualization video's FPS.
VIZ_FPS = 30#@param {type:"integer"}

In [5]:
print(THRESHOLD )

0.001


# Get input video

We provide 3 ways to get input video:
1. upload video to your Google Drive.
2. provide URL of a video.
3. record video using webcam.

In [6]:
import os
import fnmatch

# Define the directory path
dir_path = r"D:\RepNet\_twist-20230429T024838Z-001"

# Define the video file extensions to search for
video_extensions = ["*.mp4"]

# Initialize an empty list to store the video paths
video_paths = []

# Walk through the directory tree and search for video files
for root, dirs, files in os.walk(dir_path):
    
    for ext in video_extensions:
        for file in fnmatch.filter(files, ext):
            # Append the full path of the video file to the list
            video_paths.append(os.path.join(root, file))

video_paths 


['D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_103_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_105_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_108_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_117_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_124_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_127_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_130_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_136_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_139_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_144_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_161_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_162_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_166_twist.mp4',
 'D:\\RepNet\\_twist-20230429T024838Z-001\\_twist\\01_167_twist.mp4',
 'D:\\RepNet\\_twist

In [7]:
# Uncomment and run these lines to load videos from Google Drive.
# f
# drive.mount('/content/gdrive')
for i in range(len(video_paths)):
        PATH_TO_VIDEO_ON_YOUR_DRIVE = video_paths[i]
        name = os.path.basename(PATH_TO_VIDEO_ON_YOUR_DRIVE)
        name=name[:-4]
      
        input_file = PATH_TO_VIDEO_ON_YOUR_DRIVE
        cap = cv2.VideoCapture(input_file)

     
        out_dir=r"D:\RepNet\_twist-20230429T024838Z-001\fplower"
        

        # Read the video frame-by-frame and write it to the output file

        i=0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_path = os.path.join(out_dir, 'fplower_%s_%i.png'%(name,i)) # Change the extension to match your desired image format
            
            print(frame_path)
            i=i+1
            cv2.imwrite(frame_path, frame)
            if not ret:
                break
        cap.release()
       
        cv2.destroyAllWindows()

D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_0.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_1.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_2.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_3.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_4.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_5.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_6.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_7.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_8.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_9.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_10.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_11.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fplower_01_103_twist_12.png
D:\RepNet\_twist-20230429T024838Z-001\fplower\fp

In [9]:


# Open the input video file
input_file = PATH_TO_VIDEO_ON_YOUR_DRIVE
cap = cv2.VideoCapture(input_file)

# Get the video frame rate and calculate the new frame rate
fps = cap.get(cv2.CAP_PROP_FPS)


# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_file = r"D:\RepNet\_twist-20230429T024838Z-001\slow_video\%s_slow.mp4"(name)
out_dir=r"D:\RepNet\_twist-20230429T024838Z-001\fplower"
out = cv2.VideoWriter(output_file, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))


i=0
slow_down_factor=2
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        image_path = os.path.join(out_dir, 'fplower_%s_%i.png'%(name,i))
        i=i+1
        image = cv2.imread(image_path)
        for j in range(slow_down_factor):
           
            # print(image.shape)
            # Write the combined frame to the output video file
            out.write(image)
    else:
        break


# print('love') 
# Release the resources
cap.release()
out.release()
cv2.destroyAllWindows()

TypeError: 'str' object is not callable

In [10]:
for i in range(len(video_paths)):
        # Open the input video file
        PATH_TO_VIDEO_ON_YOUR_DRIVE = video_paths[i]
        name = os.path.basename(PATH_TO_VIDEO_ON_YOUR_DRIVE)
        name=name[:-4]
        input_file = PATH_TO_VIDEO_ON_YOUR_DRIVE
        cap = cv2.VideoCapture(input_file)
        # Get the video frame rate and calculate the new frame rate
        fps = cap.get(cv2.CAP_PROP_FPS)


        # Define the codec and create a VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_file = r"D:\RepNet\_twist-20230429T024838Z-001\slow_video\%s_slow.mp4"%(name)
        out = cv2.VideoWriter(output_file, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))
        i=0
        slow_down_factor=2
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                image_path = os.path.join(out_dir, 'fplower_%s_%i.png'%(name,i))
                i=i+1
                image = cv2.imread(image_path)
                for j in range(slow_down_factor):
                
                    # print(image.shape)
                    # Write the combined frame to the output video file
                    out.write(image)
            else:
                break
        cap.release()
        out.release()

# Run RepNet

In [9]:
# !pip3 install ffmpeg-python

In [31]:
# !pip3 install ffmpeg
#conda install -c conda-forge openh264

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=85dc5281aafb501db88c42f05b636d049b6a426ed5d182469d882384cdf60c9a
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\76\96\8a\d6e34080b350cd65b392d646f718891bb40157f85f4c603551
Successfully built ffmpeg


DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [1]:
import csv

# create an empty dictionary to store the data
data_dict = {}

# open the CSV file using a context manager
with open(r'D:\RepNet\FineDiving_fine-grained_annotation.csv', 'r') as csvfile:
    
    # create a csv reader object
    csvreader = csv.reader(csvfile)
    
    # skip the header row
    next(csvreader)
    
    # loop through the rows of the csv file
    for row in csvreader:
        
        # check if the value of the first column is "01"
        if row[0] == "1":
            
            # extract the values from the first, second, and sixth columns
            first_val = row[0]
            second_val = int(row[1])
            sixth_val = row[5]
            
            # check if "Soms." exists in the sixth column
            if "Soms." in sixth_val:
                
                # extract the relevant value from the sixth column
                end_index = sixth_val.find('Soms.')
                start_index = sixth_val.rfind(':', 0, end_index) + 1
                relevant_val = sixth_val[start_index:end_index]
                
                # create the key for the dictionarys
                key = '0{}_{}_somersault_label'.format(first_val, second_val)
                
                # add the key-value pair to the dictionary
                data_dict[key] = float(relevant_val)
# # rank the dictionary by second_val
# ranked_dict = dict(sorted(data_dict.items(), key=lambda item: item[1]))
# print the dictionary to check if it worked
print(data_dict["01_2_somersault_label"])



3.5


In [44]:
import csv

# create an empty dictionary to store the data
data_dict = {}

# open the CSV file using a context manager
with open(r'D:\RepNet\FineDiving_fine-grained_annotation.csv', 'r') as csvfile:
    
    # create a csv reader object
    csvreader = csv.reader(csvfile)
    
    # skip the header row
    next(csvreader)
    
    # loop through the rows of the csv file
    for row in csvreader:
        
        # check if the value of the first column is "01"
        # if row[0] == "1":
            
        # extract the values from the first, second, and sixth columns
        first_val = row[0]
        second_val = int(row[1])
        sixth_val = row[5]
        
        # check if "Soms." exists in the sixth column
        if "Soms." in sixth_val:
            
            # extract the relevant value from the sixth column
            end_index = sixth_val.find('Soms.')
            start_index = sixth_val.rfind(':', 0, end_index) + 1
            relevant_val = sixth_val[start_index:end_index]
            if len(first_val) == 1:
                first_val = "0" + first_val
            # create the key for the dictionarys
            key = '{}_{}'.format(first_val, second_val)
            key="FINADiving_MTL_256s/"+key[:key.rfind("_")] + "/" + key[key.rfind("_")+1:]
            # add the key-value pair to the dictionary
            data_dict[key] = float(relevant_val)
# # rank the dictionary by second_val
# ranked_dict = dict(sorted(data_dict.items(), key=lambda item: item[1]))
# print the dictionary to check if it worked
print(data_dict["FINADiving_MTL_256s/01/2"])
print(len(data_dict))
unique_somes = list(set(data_dict.values()))
print(unique_somes)

3.5
2780
[1.5, 2.0, 2.5, 3.5, 4.5, 3.0]


In [45]:
import csv

# create an empty dictionary to store the data
twis = {}

# open the CSV file using a context manager
with open(r'D:\RepNet\FineDiving_fine-grained_annotation.csv', 'r') as csvfile:
    
    # create a csv reader object
    csvreader = csv.reader(csvfile)
    
    # skip the header row
    next(csvreader)
    
    # loop through the rows of the csv file
    for row in csvreader:
        
        # check if the value of the first column is "01"
        # if row[0] == "1":
            
        # extract the values from the first, second, and sixth columns
        first_val = row[0]
        second_val = int(row[1])
        sixth_val = row[5]
        
        # check if "Soms." exists in the sixth column
        if "Twist"  in sixth_val:
            
            # extract the relevant value from the sixth column
            end_index = sixth_val.find('Twist')
            start_index = sixth_val.rfind(':', 0, end_index) + 1
            relevant_val = sixth_val[start_index:end_index]
            # print(relevant_val)
            if len(first_val) == 1:
                first_val = "0" + first_val
            # create the key for the dictionarys
            key = '{}_{}'.format(first_val, second_val)
            key="FINADiving_MTL_256s/"+key[:key.rfind("_")] + "/" + key[key.rfind("_")+1:]
            # add the key-value pair to the dictionary
            twis[key] = float(relevant_val)
# # rank the dictionary by second_val
# ranked_dict = dict(sorted(data_dict.items(), key=lambda item: item[1]))
# print the dictionary to check if it worked
print(twis["FINADiving_MTL_256s/01/3"])
print(len(twis))
unique_twist = list(set(twis.values()))
print(unique_twist)

1.5
801
[0.5, 1.5, 2.0, 3.0, 1.0, 2.5, 3.5]


In [7]:
import csv

# Define the input file names and the output file name
input_file_1 = r"D:\RepNet\output.txt"
input_file_2 =r"D:\RepNet\output1.txt"
output_file = r"D:\RepNet\new3.csv"

# Open the output file
with open(output_file, "w", newline='') as out_file:
    # Create a CSV writer
    csv_writer = csv.writer(out_file)
    # Write the header row
    csv_writer.writerow(["somes", "predicted_value"])
    # Open the first input file
    with open(input_file_1, "r") as in_file_1:
        # Loop over each line in the first input file
        for line in in_file_1:
            # Split the line into two values
            values = line.strip().split()
            # Write the values to the output file as a row
            csv_writer.writerow(values)
    # Open the second input file
    with open(input_file_2, "r") as in_file_2:
        # Loop over each line in the second input file
        for line in in_file_2:
            # Split the line into two values
            values = line.strip().split()
            # Write the values to the output file as a row
            csv_writer.writerow(values)

print("Conversion complete!")



Conversion complete!


In [15]:
import csv

# Define the input file name
input_file = r"D:\RepNet\predi_twist.csv"

# Create an empty dictionary to store the key-value pairs
twist= {}

# Open the input file
with open(input_file, "r") as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)
    # Loop over each row in the file
    next(csv_reader)
    for row in csv_reader:
        # Extract the key and value from the row
        key = row[0]
        key=key[:-16]
        new_key = "FINADiving_MTL_256s/"+key[:key.rfind("_")] + "/" + key[key.rfind("_")+1:]
        value = row[1]
        # Add the key-value pair to the dictionary
        twist[new_key] = value

# Print the dictionary
print(twist)
print(len(twist))

{'FINADiving_MTL_256s/18/17': '1.16', 'FINADiving_MTL_256s/FINADivingWorldCup2021_Women3m_semifinal_r5/13': '0.0', 'FINADiving_MTL_256s/FINADivingWorldCup2021_Women10m_semifinal_r5/6': '1.14', 'FINADiving_MTL_256s/17/130': '0.61', 'FINADiving_MTL_256s/26/119': '0.72', 'FINADiving_MTL_256s/FINAWorldChampionships2019_Men10mSynchronised_final_r6/6': '1.12', 'FINADiving_MTL_256s/FINAWorldChampionships2019_Men3mSynchronised_final_r4/11': '1.53', 'FINADiving_MTL_256s/FINADivingWorldCup2021_Men3mSynchronised_final_r3/6': '1.05', 'FINADiving_MTL_256s/FINADivingWorldCup2021_Women3mSynchronised_final_r5/11': '1.51', 'FINADiving_MTL_256s/FINAWorldChampionships2019_Women3mSynchronised_final_r4/1': '0.5', 'FINADiving_MTL_256s/FINAWorldChampionships2019_Women3m_final_r5/0': '0.72', 'FINADiving_MTL_256s/09/81': '0.13', 'FINADiving_MTL_256s/10/41': '0.59', 'FINADiving_MTL_256s/3mMenSpringboardFinal-EuropeanChampionships2021_4/9': '0.86', 'FINADiving_MTL_256s/13/18': '1.42', 'FINADiving_MTL_256s/FINAWo

In [14]:
import csv

# Define the input file name
input_file = r"D:\RepNet\new3.csv"

# Create an empty dictionary to store the key-value pairs
new= {}

# Open the input file
with open(input_file, "r") as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)
    # Loop over each row in the file
    next(csv_reader)
    for row in csv_reader:
        # Extract the key and value from the row
        key = row[0]
        key=key[:-16]
        new_key = "FINADiving_MTL_256s/"+key[:key.rfind("_")] + "/" + key[key.rfind("_")+1:]
        value = row[1]
        # Add the key-value pair to the dictionary
        new[new_key] = value

# Print the dictionary
print(new)
print(len(new))

{'FINADiving_MTL_256s/FINADivingWorldCup2021_Women3m_final/16': '6.58', 'FINADiving_MTL_256s/18/17': '0.72', 'FINADiving_MTL_256s/26/119': '0.75', 'FINADiving_MTL_256s/FINAWorldChampionships2019_Men10mSynchronised_final_r6/6': '0.65', 'FINADiving_MTL_256s/09/81': '2.35', 'FINADiving_MTL_256s/10/41': '1.02', 'FINADiving_MTL_256s/01/9': '1.56', 'FINADiving_MTL_256s/17/48': '1.52', 'FINADiving_MTL_256s/02/13': '3.54', 'FINADiving_MTL_256s/01/61': '1.61', 'FINADiving_MTL_256s/02/73': '4.47', 'FINADiving_MTL_256s/3mMenSpringboardFinal-EuropeanChampionships2021_3/21': '4.57', 'FINADiving_MTL_256s/FINADivingWorldCup2021_Men10mSynchronised_final_r5/3': '3.67', 'FINADiving_MTL_256s/FINADivingWorldCup2021_Men3m_semifinal_r2/17': '3.64', 'FINADiving_MTL_256s/01/184': '2.6', 'FINADiving_MTL_256s/FullMenSynchronised10mPlatform_Tokyo2020Replays_3/0': '4.95', 'FINADiving_MTL_256s/FINADivingWorldCup2021_Women3mSynchronised_final_r2/3': '3.6', 'FINADiving_MTL_256s/01/185': '1.07', 'FINADiving_MTL_256s/

In [25]:
import pandas as pd

# read the existing CSV file into a DataFrame
df = pd.read_excel(r'D:\RepNet\lr_dataset(ST).xlsx')
print(df.head(100))
# define the two dictionaries
dict1 = new
dict2 = twist

# iterate over each row of the DataFrame and update with the values from the dictionaries
for index, row in df.iterrows():
    key = row.iloc[0]  # get the value from the first column of the row
    if key in dict1:
        df.loc[index, 'pred_soms.'] = dict1[key]
    if key in dict2:
        df.loc[index, 'pred_twist'] = dict2[key]
print(len(df))

# # write the updated DataFrame to the existing CSV file
df.to_excel(r'D:\RepNet\lr_dataset(ST).xlsx', index=False)


                                              vidpath  actualscore  predscore
0                           FINADiving_MTL_256s/01/29     0.642857   0.597425
1   FINADiving_MTL_256s/FINADivingWorldCup2021_Wom...     0.439024   0.538997
2                           FINADiving_MTL_256s/05/34     0.627178   0.609707
3                           FINADiving_MTL_256s/07/31     0.372387   0.526531
4                           FINADiving_MTL_256s/26/87     0.423345   0.595931
..                                                ...          ...        ...
95                          FINADiving_MTL_256s/26/39     0.524390   0.609219
96  FINADiving_MTL_256s/FINADivingWorldCup2021_Men...     0.710801   0.590349
97  FINADiving_MTL_256s/FINAWorldChampionships2019...     0.613240   0.607397
98  FINADiving_MTL_256s/FINADivingWorldCup2021_Wom...     0.512195   0.561492
99  FINADiving_MTL_256s/FINADivingWorldCup2021_Men...     0.676829   0.594664

[100 rows x 3 columns]
2218


In [49]:
import pandas as pd

# define the unique values
unique_somes = [1.5, 2.0, 2.5, 3.5, 4.5, 3.0]
unique_twist = [0.5, 1.5, 2.0, 3.0, 1.0, 2.5, 3.5]

# read the Excel file into a pandas dataframe
df = pd.read_excel('D:\RepNet\lr_dataset(ST).xlsx')

# apply the changes to the "pred_soms." column
df['modify_soms.'] = df['pred_soms.'].apply(lambda x: min(unique_somes, key=lambda y: abs(y-x)) if pd.notna(x) else x)

# apply the changes to the "pred_twist" column
df['modify_twist'] = df['pred_twist'].apply(lambda x: min(unique_twist, key=lambda y: abs(y-x)) if pd.notna(x) else x)
print(df.head(100))
print(len(df))
# write the updated dataframe back to the Excel file
df.to_excel('D:\RepNet\lr_dataset(ST).xlsx', index=False)

                                              vidpath  actualscore  predscore  \
0                           FINADiving_MTL_256s/01/29     0.642857   0.597425   
1   FINADiving_MTL_256s/FINADivingWorldCup2021_Wom...     0.439024   0.538997   
2                           FINADiving_MTL_256s/05/34     0.627178   0.609707   
3                           FINADiving_MTL_256s/07/31     0.372387   0.526531   
4                           FINADiving_MTL_256s/26/87     0.423345   0.595931   
..                                                ...          ...        ...   
95                          FINADiving_MTL_256s/26/39     0.524390   0.609219   
96  FINADiving_MTL_256s/FINADivingWorldCup2021_Men...     0.710801   0.590349   
97  FINADiving_MTL_256s/FINAWorldChampionships2019...     0.613240   0.607397   
98  FINADiving_MTL_256s/FINADivingWorldCup2021_Wom...     0.512195   0.561492   
99  FINADiving_MTL_256s/FINADivingWorldCup2021_Men...     0.676829   0.594664   

    pred_soms.  pred_twist 

In [43]:
import pandas as pd

# read the existing CSV file into a DataFrame
df = pd.read_excel(r'D:\RepNet\lr_dataset(ST).xlsx')
print(df.head(100))
# define the two dictionaries
dict1 = data_dict
dict2 = twis

# iterate over each row of the DataFrame and update with the values from the dictionaries
for index, row in df.iterrows():
    key = row.iloc[0]  # get the value from the first column of the row
    if key in dict1:
        df.loc[index, 'fine_grained_soms.'] = dict1[key]
    if key in dict2:
        df.loc[index, 'fine_grained_twist'] = dict2[key]
print(len(df))

# # write the updated DataFrame to the existing CSV file
df.to_excel(r'D:\RepNet\lr_dataset(ST).xlsx', index=False)

                                              vidpath  actualscore  predscore  \
0                           FINADiving_MTL_256s/01/29     0.642857   0.597425   
1   FINADiving_MTL_256s/FINADivingWorldCup2021_Wom...     0.439024   0.538997   
2                           FINADiving_MTL_256s/05/34     0.627178   0.609707   
3                           FINADiving_MTL_256s/07/31     0.372387   0.526531   
4                           FINADiving_MTL_256s/26/87     0.423345   0.595931   
..                                                ...          ...        ...   
95                          FINADiving_MTL_256s/26/39     0.524390   0.609219   
96  FINADiving_MTL_256s/FINADivingWorldCup2021_Men...     0.710801   0.590349   
97  FINADiving_MTL_256s/FINAWorldChampionships2019...     0.613240   0.607397   
98  FINADiving_MTL_256s/FINADivingWorldCup2021_Wom...     0.512195   0.561492   
99  FINADiving_MTL_256s/FINADivingWorldCup2021_Men...     0.676829   0.594664   

    pred_soms.  pred_twist 

In [5]:
with open('D:\RepNet\output.txt', 'w') as file:
    for key, value in data_dict.items():
        file.write(key + ' ' + str(value) + '\n')

In [14]:
PATH_TO_VIDEO_ON_YOUR_DRIVE = video_paths[0]
name = os.path.basename(PATH_TO_VIDEO_ON_YOUR_DRIVE)
name=name[:-4]
imgs, vid_fps = read_video(video_paths[0])
print('Running RepNet...') 
(pred_period, pred_score, within_period,
per_frame_counts, chosen_stride) = get_counts(
    model,
    imgs,
    strides=[1,2,3,4],
    batch_size=20,
    threshold=THRESHOLD,
    within_period_threshold=WITHIN_PERIOD_THRESHOLD,
    constant_speed=CONSTANT_SPEED,
    median_filter=MEDIAN_FILTER,
    fully_periodic=FULLY_PERIODIC)
print('Visualizing results...') 
frames=imgs
count=per_frame_counts
if isinstance(count, list):
  counts = len(frames) * [count/len(frames)]
else:
  counts = count
sum_counts = np.cumsum(counts)
print(sum_counts[-1])

Running RepNet...
Visualizing results...
0.776785746216774


In [19]:
pred_twist={}
for i in range(len(video_paths)):
        PATH_TO_VIDEO_ON_YOUR_DRIVE = video_paths[i]
        name = os.path.basename(PATH_TO_VIDEO_ON_YOUR_DRIVE)
        name=name[:-4]
        imgs, vid_fps = read_video(video_paths[i])
        print('Running RepNet...') 
        (pred_period, pred_score, within_period,
        per_frame_counts, chosen_stride) = get_counts(
            model,
            imgs,
            strides=[1,2,3,4],
            batch_size=20,
            threshold=THRESHOLD,
            within_period_threshold=WITHIN_PERIOD_THRESHOLD,
            constant_speed=CONSTANT_SPEED,
            median_filter=MEDIAN_FILTER,
            fully_periodic=FULLY_PERIODIC)
        print('Visualizing results...') 
        frames=imgs
        count=per_frame_counts
        if isinstance(count, list):
           counts = len(frames) * [count/len(frames)]
        else:
           counts = count
        sum_counts = np.cumsum(counts)
        print(sum_counts[-1])
        key = '%stwist_pred'%(name)
        pred_twist[key] = round(float(sum_counts[-1]),2)
        with open(r'D:\RepNet\predi_twist.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for key, value in pred_twist.items():
                writer.writerow([key, value])
                # viz_reps(imgs, per_frame_counts, pred_score,name, interval=1000/VIZ_FPS,
        #         plot_score=PLOT_SCORE)


# pred_soms = {'ID': 'pred_soms '}




Running RepNet...
Visualizing results...
0.776785746216774
Running RepNet...
Visualizing results...
0.5000000037252903
Running RepNet...
Visualizing results...
0.8883928954601288
Running RepNet...
Visualizing results...
1.4375
Running RepNet...
Visualizing results...
1.0069444477558136
Running RepNet...
Visualizing results...
1.2876984532922506
Running RepNet...
Visualizing results...
1.4732142984867096
Running RepNet...
Visualizing results...
0.6666666716337204
Running RepNet...
Visualizing results...
0.9375000037252903
Running RepNet...
Visualizing results...
1.3707071095705032
Running RepNet...
Visualizing results...
0.6666666716337204
Running RepNet...
Visualizing results...
0.7817460428923368
Running RepNet...
Visualizing results...
1.013888891786337
Running RepNet...
Visualizing results...
1.0347222238779068
Running RepNet...
Visualizing results...
1.0000000186264515
Running RepNet...
Visualizing results...
1.4191919602453709
Running RepNet...
Visualizing results...
1.00000001862

c:\Users\lenovo\anaconda3\envs\sifaa\lib\site-packages\ipykernel_launcher.py:898: RuntimeWarning: divide by zero encountered in double_scalars


No repetitions detected in video as score 0.00 is less than threshold 0.00.
Visualizing results...
0.0
Running RepNet...
Visualizing results...
0.7195767313241959
Running RepNet...
Visualizing results...
0.6111111156642437
Running RepNet...
Visualizing results...
0.1750000026077032
Running RepNet...
Visualizing results...
0.649305559694767
Running RepNet...
Visualizing results...
0.8888889066874981
Running RepNet...
Visualizing results...
0.12962963059544563
Running RepNet...
Visualizing results...
1.2645202353596687
Running RepNet...
Visualizing results...
0.6666666716337204
Running RepNet...
Visualizing results...
0.2222222238779068
Running RepNet...
Visualizing results...
1.0312500447034836
Running RepNet...
Visualizing results...
1.0793650969862938
Running RepNet...
Visualizing results...
0.95238097012043
Running RepNet...
Visualizing results...
0.4618055559694767
Running RepNet...
Visualizing results...
0.14814814925193787
Running RepNet...
Visualizing results...
0.444444447755813

In [51]:
import re
import numpy as np
import matplotlib.pyplot as plt

# First dictionary
dict1 = pred_soms

# Second dictionary
dict2 = data_dict

# Extracting the string before the first "s" in each key of the first dictionary
dict1_str = {}
for key, value in dict1.items():
    str_before_s = re.match(r"(.*?)s", key).group(1)
    dict1_str[str_before_s] = value
print(dict1_str["01_55_"])
# Extracting the string before the first "s" in each key of the second dictionary
dict2_str = {}
for key, value in dict2.items():
    str_before_s = re.match(r"(.*?)s", key).group(1)
    dict2_str[str_before_s] = value
print(np.array(dict2_str['01_55_']))
# # Matching the strings in the first and second dictionaries and calculating the MSE for each pair
mse_dict = {}
s={}
d={}
for key1, value1 in dict1_str.items():
    if key1 in dict2_str:
        value2 = dict2_str[key1]
        error =abs((value1- value2)/value2)
        # error = (np.array(value1) - np.array(value2))**2
        print(key1)
        print(error)
        mse_dict[key1] = error
        d[key1] = value1
values_list = list(mse_dict.values())

ME=np.mean(np.array(values_list))
print(ME)



5.87
3.5
01_53_
0.0
01_67_
0.13666666666666671
01_14_
0.2457142857142857
01_23_
0.03714285714285711
01_65_
0.5942857142857143
01_26_
0.25428571428571417
01_1_
0.0942857142857143
01_25_
0.65
01_63_
0.6457142857142857
01_45_
0.6759999999999999
01_7_
0.10571428571428575
01_12_
0.0971428571428571
01_21_
0.051111111111111204
01_48_
0.10000000000000002
01_70_
0.029999999999999954
01_47_
0.4371428571428572
01_54_
0.665
01_78_
0.14857142857142844
01_92_
0.665
01_28_
0.4542857142857143
01_79_
0.482857142857143
01_97_
0.6342857142857142
01_105_
0.605
01_32_
0.029999999999999954
01_71_
0.014285714285714235
01_76_
0.25714285714285723
01_94_
0.0
01_11_
0.034285714285714315
01_8_
0.355
01_43_
0.43142857142857144
01_85_
0.36666666666666675
01_59_
0.23333333333333328
01_84_
0.22285714285714292
01_17_
0.03200000000000003
01_83_
0.6799999999999999
01_103_
0.63
01_49_
0.26571428571428574
01_89_
0.28888888888888886
01_88_
0.16
01_95_
0.0285714285714286
01_37_
0.4566666666666667
01_69_
0.13142857142857142


In [52]:
# with open(r'C:\Users\lenovo\Desktop\RepNet\new.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     for key, value in mse_dict.items():
#         writer.writerow([key, value])
with open(r'C:\Users\lenovo\Desktop\RepNet\new1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in d.items():
        writer.writerow([key, value])

In [14]:
print('Running RepNet...') 
imgs, vid_fps = read_video(r'E:\itsspm\itsspm\models\slow_video\01_1_somersault_slow.mp4')
(pred_period, pred_score, within_period,
 per_frame_counts, chosen_stride) = get_counts(
     model,
     imgs,
     strides=[1,2,3,4],
     batch_size=20,
     threshold=THRESHOLD,
     within_period_threshold=WITHIN_PERIOD_THRESHOLD,
     constant_speed=CONSTANT_SPEED,
     median_filter=MEDIAN_FILTER,
     fully_periodic=FULLY_PERIODIC)
print('Visualizing results...') 
frames=imgs
count=per_frame_counts
if isinstance(count, list):
    counts = len(frames) * [count/len(frames)]
else:
    counts = count
sum_counts = np.cumsum(counts)
print(sum_counts[-1])
# print(pred_period, pred_score, within_period,
#  per_frame_counts, chosen_stride)
# viz_reps(imgs, per_frame_counts, pred_score, interval=1000/VIZ_FPS,
#          plot_score=PLOT_SCORE)


Running RepNet...
Visualizing results...
3.347222238779068


# Evaluation

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Read the Excel file into a pandas DataFrame
df = pd.read_excel('D:\RepNet\lr_dataset (new).xlsx')
df.dropna(inplace=True)
# Get the predicted and ground truth values
pred_soms = df['pred_soms.']
pred_twist = df['pred_twist']
gt_soms = df['fine_grained_soms.']
gt_twist = df['fine_grained_twist']
modify_soms = df['modify_soms.']
modify_twist = df['modify_twist']

# Calculate the evaluation metrics for pred_soms
mae_soms = mean_absolute_error(gt_soms, pred_soms)
mse_soms = mean_squared_error(gt_soms, pred_soms)

# Calculate the evaluation metrics for modify_soms
mae_modify_soms = mean_absolute_error(gt_soms, modify_soms)
mse_modify_soms = mean_squared_error(gt_soms, modify_soms)

# Calculate the evaluation metrics for pred_twist
mae_twist = mean_absolute_error(gt_twist, pred_twist)
mse_twist = mean_squared_error(gt_twist, pred_twist)

# Calculate the evaluation metrics for modify_twist
mae_modify_twist = mean_absolute_error(gt_twist, modify_twist)
mse_modify_twist = mean_squared_error(gt_twist, modify_twist)

# Print the evaluation results

# Print the evaluation results for pred_soms
print("MAE for pred_soms:")
print(mae_soms)
print("MSE for pred_soms:")
print(mse_soms)

# Calculate the evaluation metrics for pred_soms
mae_soms = mean_absolute_error(gt_soms, pred_soms)
mse_soms = mean_squared_error(gt_soms, pred_soms)

# Calculate the evaluation metrics for modify_soms
mae_modify_soms = mean_absolute_error(gt_soms, modify_soms)
mse_modify_soms = mean_squared_error(gt_soms, modify_soms)

# Calculate the evaluation metrics for pred_twist
mae_twist = mean_absolute_error(gt_twist, pred_twist)
mse_twist = mean_squared_error(gt_twist, pred_twist)

# Calculate the evaluation metrics for modify_twist
mae_modify_twist = mean_absolute_error(gt_twist, modify_twist)
mse_modify_twist = mean_squared_error(gt_twist, modify_twist)

# Create a table of evaluation results
evaluation_table = pd.DataFrame({
    'Metrics': ['MAE', 'MSE'],
    'pred_soms': [mae_soms, mse_soms],
    'modify_soms': [mae_modify_soms, mse_modify_soms],
    'pred_twist': [mae_twist, mse_twist],
    'modify_twist': [mae_modify_twist, mse_modify_twist]
})

# Save the evaluation table to a CSV file
evaluation_table.to_csv(r'D:\RepNet\evaluation_results.csv', index=False)

# Create scatter plots of the predicted vs. ground truth soms values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(gt_soms, pred_soms)
plt.xlabel('Ground Truth soms')
plt.ylabel('Predicted soms')
plt.title('Predicted vs. Ground Truth soms')

plt.subplot(1, 2, 2)
plt.scatter(gt_soms, modify_soms)
plt.xlabel('Ground Truth soms')
plt.ylabel('Modify soms')
plt.title('Modify vs. Ground Truth soms')

plt.tight_layout()
plt.savefig(r'D:\RepNet\soms_scatter_plots.png')
plt.close()

# Create scatter plots of the predicted vs. ground truth twist values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(gt_twist, pred_twist)
plt.xlabel('Ground Truth twist')
plt.ylabel('Predicted twist')
plt.title('Predicted vs. Ground Truth twist')

plt.subplot(1, 2, 2)
plt.scatter(gt_twist, modify_twist)
plt.xlabel('Ground Truth twist')
plt.ylabel('Modify twist')
plt.title('Modify vs. Ground Truth twist')

plt.tight_layout()
plt.savefig(r'D:\RepNet\twist_scatter_plots.png')
plt.close()


MAE for pred_soms:
1.3664622641509434
MSE for pred_soms:
2.054227515723271


# Citation

If you found our paper/code useful in your research, consider citing our paper:


```
@InProceedings{Dwibedi_2020_CVPR,
author = {Dwibedi, Debidatta and Aytar, Yusuf and Tompson, Jonathan and Sermanet, Pierre and Zisserman, Andrew},
title = {Counting Out Time: Class Agnostic Video Repetition Counting in the Wild},
booktitle = {IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR)},
month = {June},
year = {2020}
}
```



